## Delivery Market Analysis on Price

- Importing the necessary libraries

In [1]:
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd


Combine the relevant fields of all three databases into one table inside a separate database (to keep it organized).

In [2]:
# Create a new bd to combine the relevant info from all three databases, conenct and attach connection to other databases, create table
conn = sqlite3.connect('./databases/combined.db')
conn.execute("ATTACH DATABASE './databases/deliveroo.db' AS del")
conn.execute("ATTACH DATABASE './databases/takeaway.db' AS take")
conn.execute("ATTACH DATABASE './databases/ubereats.db' AS uber")

# create new table to combine all platforms - rest_id to text due to Takeaway ID format
cur = conn.cursor()
cur.execute("""
    CREATE TABLE main.restaurants (
        id INTEGER PRIMARY KEY,
        platform TEXT,
        rest_id TEXT,
        rest_name TEXT,
        rating REAL,
        rating_count INTEGER,
        delivery_fee REAL,
        city TEXT,
        zip_code INTEGER,
        latitude REAL,
        longitude REAL,         
        item_id INTEGER,
        item_name TEXT,
        price REAL
    )
""")

- Add the data from each app and fill the new 'platform' column for each app.

In [3]:
# Add data from Deliveroo
cur.execute("""
    INSERT INTO main.restaurants (rest_id, rest_name, rating, rating_count, delivery_fee, zip_code,  latitude, longitude, item_id, item_name, price)
    SELECT r.id, r.name, r.rating, r.rating_number, r.delivery_fee, postal_code, latitude, longitude,  m.id, m.name, price
    FROM del.restaurants AS r
    INNER JOIN del.menu_items AS m
    ON r.id = m.restaurant_id
""")

# Fill platform column - Deliveroo
cur.execute("""
    UPDATE main.restaurants
    SET platform = 'Deliveroo';
""")

# Add data from Takeaway
cur.execute("""
    INSERT INTO main.restaurants (rest_id, rest_name, rating, rating_count, delivery_fee, city, latitude, longitude, item_id, item_name, price)
    SELECT r.restaurant_id, r.name, ratings, ratingsNumber, deliveryFee, city, latitude, longitude, m.id, m.name, price
    FROM take.restaurants AS r
    INNER JOIN take.menuItems AS m
    USING(primarySlug)           
""")

# Fill platform column - Takeaway
cur.execute("""
    UPDATE main.restaurants
    SET platform = 'Takeaway'
    WHERE platform IS NULL;
""")

# Add data from Uber Eats (extract zip zode from location__address col)
cur.execute("""
    INSERT INTO main.restaurants (rest_id, rest_name, rating, rating_count, city, zip_code, latitude, longitude, item_id, item_name, price)
    SELECT r.id, r.title, r.rating__rating_value, r.rating__review_count, location__city, substr(location__address, -4, 4), location__latitude, location__longitude, m.id, m.name, m.price
    FROM uber.restaurants AS r
    INNER JOIN uber.menu_items AS m
    ON r.id = m.restaurant_id;
""")

# Fill platform column - Uber Eats
cur.execute("""
    UPDATE main.restaurants
    SET platform = 'UberEats'
    WHERE platform IS NULL;
""")

Now that we have the information from the three platforms into one, we can start exploring and cleaning the data.

In [4]:
# Delete negative sign from price
cur.execute("""
    UPDATE main.restaurants
    SET price = REPLACE(price, '-', '')
""")

# Delete + sig from rating count
cur.execute("""
    UPDATE main.restaurants
    SET rating_count = REPLACE(rating_count, '+', '')
""")

# Remove rows without price
cur.execute("""
    DELETE
    FROM main.restaurants
    WHERE price = 0.0
""")

# Change data type to INTEGER for rating_count and zip_code
# rating_count
cur.execute("""
    UPDATE main.restaurants
    SET rating_count = CAST(rating_count AS INTEGER)
    WHERE typeof(rating_count) = 'text';
""")
# zip_code
cur.execute("""
    UPDATE main.restaurants
    SET zip_code = CAST(zip_code AS INTEGER)
    WHERE typeof(zip_code) = 'text';
""")

# Correct prices for Uber Eats (after verifying samples on the Uber Eats website)
cur.execute("""
    UPDATE main.restaurants
    SET price = price / 100
    WHERE platform = 'UberEats';
""")

After further exploring the data, the following is noticeable about extra high and low prices: <br>
- Prices over 100 correspond to expensive champagne or wine bottles, going up to about EUR 250. <br>
- Some high prices were simply incorrect eg. Vegetable Samosa with a price of EUR 600.0 <br>
- Some low prices as well, like a Chocolate Rainbow	donut from Dunkin Donuts for EUR 0.10 (although we wish it was the actual price). <br><br>
Chamgpagne and wine bottles were kept in the table, since they represent actual products available for purchase. <br>
The others were removed:




In [5]:
cur.execute("""
    DELETE
    FROM main.restaurants
    WHERE price > 250
    OR price < 0.15;
""")

### Price related analysis
### - What is the price distribution of menu items?

In [6]:
# Get the min, max, range, and average price for each platform
price_dist_query = cur.execute("""
SELECT 
    platform,
    MIN(price) AS min_price,
    MAX(price) AS max_price,
    MAX(price) - MIN(price) AS price_range, 
    ROUND(AVG(price), 2) AS avg_price
FROM main.restaurants
GROUP BY platform
ORDER BY avg_price, price_range;
            """)

In [7]:
price_dist_data = price_dist_query.fetchall()
price_dist_data = pd.DataFrame(price_dist_data, columns=[x[0] for x in price_dist_query.description])

price_dist_long = price_dist_data.melt(
    id_vars="platform",  
    var_name="metric",   
    value_name="value" 
).sort_values(by='platform')

# Plot min price per platform
min_fig = px.bar(price_dist_data, 
             x="platform", 
             y="min_price", 
             orientation='v',
             text = 'min_price',
             hover_data=["platform", "min_price"],
             height = 500,
             width = 500,
             title="Minimum item price by Platform",
             labels={"platform": "Platform", "min_price": "Min price", },
            color_discrete_sequence=["#55828b"]
)

# Change how info shows when hoovering over the bars
min_fig.update_traces(
    hovertemplate="<b>%{x}</b><br>Minimum Price: € %{y:.2f}<extra></extra>",  
    texttemplate="€%{text:.2f}", 
    textposition="outside"  
)
min_fig.show()

# Plot max price per platform
max_fig = px.bar(price_dist_data, 
             x="platform", 
             y="max_price", 
             orientation='v',
             text = 'max_price',
             hover_data=["platform", "max_price"],
             height = 500,
             width = 500,
             title="Maximum item price by Platform",
             labels={"platform": "Platform", "max_price": "Max price", },
             color_discrete_sequence=["#87bba2"],
             color="platform", 

             color_discrete_map={
            "Uber": "#87bba2",      
            "Takeaway": "#87bba2",   
            "Deliveroo": "#55828b"   
            }
)

max_fig.update_traces(
    hovertemplate="<b>%{x}</b><br>Maximum Price: € %{y:.2f}<extra></extra>",  
    texttemplate="€%{text:.2f}",  
    textposition="outside"  
)
max_fig.show()

# Plot avg price per platform
avg_fig = px.bar(
    price_dist_data,
    x="platform",
    y="avg_price",
    orientation='v',
    text="avg_price",  
    hover_data={"platform": False, "avg_price": False},  
    height=500,
    width=500,
    title="Average Item Price by Platform",
    labels={"platform": "Platform", "avg_price": "Average Price"},
    color="platform", 
    color_discrete_sequence=["#87bba2"], 
    color_discrete_map={
        "Uber": "55828b",      
        "Takeaway": "#55828b",   
        "Deliveroo": "#87bba2"   
    }
)

avg_fig.update_traces(
    hovertemplate="<b>%{x}</b><br>Average Price: € %{y:.2f}<extra></extra>",  
    texttemplate="€%{text:.2f}", 
    textposition="outside" 
)
avg_fig.show()


### - Which restaurants have the best price-to-rating ratio? <br>

- Best overall:

In [8]:
best_restaurant = cur.execute("""
SELECT platform, rest_name, rating,
ROUND(AVG(price), 2) AS avg_price
FROM main.restaurants
WHERE rating_count >= 10
GROUP BY rest_id 
ORDER BY rating DESC, avg_price ASC
LIMIT 1;
            """).fetchall()
best_restaurant

[('Takeaway', 'QueTacos', 5.0, 2.86)]

- Top per platform:

In [9]:
best_per_platform_query = """
WITH ranked_restaurants AS (
    SELECT platform, rest_name, rating, 
            ROUND(AVG(price), 2) AS avg_price,
            ROW_NUMBER() OVER (PARTITION BY platform ORDER BY rating DESC, ROUND(AVG(price), 2) ASC) AS rank
    FROM main.restaurants
    WHERE rating_count >= 10
    GROUP BY platform, rest_id, rest_name, rating
)
SELECT platform, rest_name, rating, avg_price
FROM ranked_restaurants
WHERE rank = 1;
"""

best_per_platform_df = pd.DataFrame(cur.execute(best_per_platform_query).fetchall(), columns=[x[0] for x in cur.execute(best_per_platform_query).description])

best_per_platform_long = best_per_platform_df.melt(
    id_vars=["platform", 'rest_name'], 
    var_name="metric",   
    value_name="value"   
).sort_values(by='platform')

best_per_platform_long['platform_rest_name'] = (best_per_platform_long['platform'] + " - " + best_per_platform_long['rest_name'])

In [10]:
best_per_platform_fig = px.bar(
    best_per_platform_long,
    height=500,
    width=500,
    orientation='v',
    x='platform_rest_name',
    y="value",
    color="metric",  
    barmode="group",  
    title="Best price-rating restaurants by Platform",
    labels={"value": "Average item price and rating", "platform_rest_name": "Best restaurant per platform"},
    text='value',
    hover_data={"platform_rest_name": True, "metric": True, 'value': True},
    color_discrete_sequence='metric',
        color_discrete_map={     
        "avg_price": "#55828b",   
        "rating": "#87bba2"   
    }
)


best_per_platform_fig.update_traces(
    hovertemplate="<b>%{x}</b><br>%{y:.2f}<extra></extra>",  
    texttemplate="%{text:.2f}", 
    textposition="inside" 
)
best_per_platform_fig.show()

- Top 3 restaurants per platform

In [11]:
# Deliveroo
best_deliveroo = cur.execute("""
SELECT rest_name, rating,
ROUND(AVG(price), 2) AS avg_price
FROM main.restaurants
WHERE rating_count >= 10
AND platform = 'Deliveroo'
GROUP BY rest_id 
ORDER BY rating DESC, avg_price ASC
LIMIT 3;
            """)
best_deliveroo_df = pd.DataFrame(best_deliveroo.fetchall(), columns=['Restaurant', 'Rating', 'Avg. item price €'])


# plot
best_takeaway_df_long = best_deliveroo_df.melt(
    id_vars=["Restaurant"], 
    var_name="metric",   
    value_name="value"   
).sort_values(by='Restaurant')

best_takeaway_df_long_fig = px.bar(
    best_takeaway_df_long ,
    height=500,
    width=500,
    orientation='v',
    x='Restaurant',
    y="value",
    color="metric",  
    barmode="group",  
    title="Best price-rating restaurants - Deliveroo",
    labels={"value": "Average item price and rating", "platform_rest_name": "Best restaurant per platform"},
    text='value',
    hover_data={"Restaurant": True, "metric": True, 'value': True},
    color_discrete_sequence=['#87bba2', "#55828b"],
)

best_takeaway_df_long_fig.update_traces(
    hovertemplate="<b>%{x}</b><br>%{y:.2f}<extra></extra>",  
    texttemplate="%{text:.2f}", 
    textposition="inside" 
)
best_takeaway_df_long_fig.show()


In [12]:
# Takeaway
best_takeaway =cur.execute("""
SELECT rest_name, rating,
ROUND(AVG(price), 2) AS avg_price
FROM main.restaurants
WHERE rating_count >= 10
AND platform = 'Takeaway'
GROUP BY rest_id 
ORDER BY rating DESC, avg_price ASC
LIMIT 3;
            """)

best_takeaway_df = pd.DataFrame(best_takeaway.fetchall(), columns=['Restaurant', 'Rating', 'Avg. item price €'])

# plot
best_takeaway_df_long = best_takeaway_df.melt(
    id_vars=["Restaurant"], 
    var_name="metric",   
    value_name="value"   
).sort_values(by='Restaurant')


best_takeaway_df_fig = px.bar(
    best_takeaway_df_long ,
    height=500,
    width=500,
    orientation='v',
    x='Restaurant',
    y="value",
    color="metric",  
    barmode="group",  
    title="Best price-rating restaurants - Takeaway",
    labels={"value": "Average item price and rating", "platform_rest_name": "Best restaurant per platform"},
    text='value',
    hover_data={"Restaurant": True, "metric": True, 'value': True},
    color_discrete_sequence=['#87bba2', "#55828b"],
)


best_takeaway_df_fig.update_traces(
    hovertemplate="<b>%{x}</b><br>%{y:.2f}<extra></extra>",  
    texttemplate="%{text:.2f}", 
    textposition="inside" 
)
best_takeaway_df_fig.show()

In [13]:
# Uber Eats
best_uber = cur.execute("""
SELECT rest_name, rating,
ROUND(AVG(price), 2) AS avg_price
FROM main.restaurants
WHERE rating_count >= 10
AND platform = 'UberEats'
GROUP BY rest_id 
ORDER BY rating DESC, avg_price ASC
LIMIT 3;
            """)

best_uber_df = pd.DataFrame(best_uber.fetchall(), columns=['Restaurant', 'Rating', 'Avg. item price €'])


# plot
best_takeaway_df_long = best_uber_df.melt(
    id_vars=["Restaurant"], 
    var_name="metric",   
    value_name="value"   
).sort_values(by='Restaurant')


best_takeaway_df_fig = px.bar(
    best_takeaway_df_long ,
    height=500,
    width=500,
    orientation='v',
    x='Restaurant',
    y="value",
    color="metric",  
    barmode="group",  
    title="Best price-rating restaurants - Uber Eats",
    labels={"value": "Average item price and rating", "platform_rest_name": "Best restaurant per platform"},
    text='value',
    hover_data={"Restaurant": True, "metric": True, 'value': True},
    color_discrete_sequence=['#87bba2', "#55828b"],
)


best_takeaway_df_fig.update_traces(
    hovertemplate="<b>%{x}</b><br>%{y:.2f}<extra></extra>",  
    texttemplate="%{text:.2f}", 
    textposition="inside" 
)
best_takeaway_df_fig.show()

- Best price-ratio restaurant with 500+ reviews:

In [14]:
# Uber Eats
best_500 = cur.execute("""
SELECT platform, rest_name, 
'Rating: ' || rating, 
'Rating count: ' || CAST(rating_count AS INTEGER),
'€ ' || ROUND(AVG(price), 2) AS avg_price
FROM main.restaurants
WHERE rating_count >= 500
GROUP BY rest_id 
ORDER BY rating DESC, avg_price ASC
;
            """).fetchone()
best_500

('Takeaway',
 'De Echte Eethuis Carlos',
 'Rating: 5.0',
 'Rating count: 599',
 '€ 11.08')

### - How do delivery fees vary across platforms and locations? <br>
The Uber Eats database does not contain information on the delivery fee, but we can still check the delivery fees for Deliveroo and Takeaway:

In [15]:
ave_del_fees = cur.execute("""
SELECT platform, 'Avg. delivery fee: € ' || ROUND(AVG(delivery_fee), 2)
FROM main.restaurants
WHERE delivery_fee > 0
GROUP BY platform
""").fetchall()
print(ave_del_fees)

query = """
SELECT platform, ROUND(AVG(delivery_fee), 2)
FROM main.restaurants
WHERE delivery_fee > 0
GROUP BY platform
"""

ave_del_fees_df = pd.DataFrame(cur.execute(query))
ave_del_fees_df

ave_del_fees_fig = px.bar(ave_del_fees_df, 
             x=1, 
             y=0, 
             orientation='h',
             text = 1,
             hover_data=[0, 1],
             height = 500,
             width = 500,
             title="Average delivery fee by platform",
             labels={"0": "Platform", "1": "Average delivery fee", },
             color_discrete_sequence=["#87bba2"],
             color=0, 
             color_discrete_map={
            0: "#87bba2",      
            1: "#87bba2",   
            "Deliveroo": "#55828b"   
            }
)

ave_del_fees_fig.update_traces(
    hovertemplate="<b>%{x}</b><br>Maximum Price: € %{y:.2f}<extra></extra>",  
    texttemplate="€%{text:.2f}",  
    textposition="inside"  
)
ave_del_fees_fig.show()

[('Deliveroo', 'Avg. delivery fee: € 4.98'), ('Takeaway', 'Avg. delivery fee: € 3.92')]


In [16]:
min_fees = cur.execute("""
SELECT platform, MIN(delivery_fee)
FROM main.restaurants
WHERE delivery_fee > 0
GROUP BY platform;
""").fetchall()

max_fees = cur.execute("""
SELECT platform, MAX(delivery_fee)
FROM main.restaurants
WHERE delivery_fee > 0
GROUP BY platform;
""").fetchall()

min_fees_df = pd.DataFrame(min_fees)
max_fees_df = pd.DataFrame(max_fees)

min_fees_fig = px.bar(min_fees_df, 
             x=1, 
             y=0, 
             orientation='h',
             text = 1,
             hover_data=[1],
             height = 500,
             width = 500,
             title="Minimum delivery fee per item",
             labels={"0": "Platform", "1": " delivery fee", },
             color_discrete_sequence=["#87bba2"],
             color=0, 
             color_discrete_map={
            0: "#87bba2",      
            1: "#87bba2",   
            "Deliveroo": "#55828b"   
            }
)

min_fees_fig.update_traces(
    hovertemplate="<b>%{x}</b><br>Min fee: € %{x:.2f}<extra></extra>",  
    texttemplate="€%{text:.2f}",  
    textposition="outside"  
)
min_fees_fig.show()

max_fees_fig = px.bar(max_fees_df, 
             x=1, 
             y=0, 
             orientation='h',
             text = 1,
             hover_data=[1],
             height = 500,
             width = 500,
             title="Maximum delivery fee per item",
             labels={"0": "Platform", "1": " delivery fee", },
             color_discrete_sequence=["#87bba2"],
             color=0, 
             color_discrete_map={
            0: "#87bba2",      
            1: "#87bba2",   
            "Deliveroo": "#55828b"   
            }
)

max_fees_fig.update_traces(
    hovertemplate="<b>%{x}</b><br>Max fee: € %{x:.2f}<extra></extra>",  
    texttemplate="€%{text:.2f}",  
    textposition="inside"  
)
max_fees_fig.show()

- Delivery fees by location 

In [17]:
cur.execute("""SELECT city, ROUND(AVG(delivery_fee), 2) AS avg_fee
            FROM main.restaurants
            WHERE city IS NOT NULL AND delivery_fee > 0
            GROUP BY city
            ORDER by avg_fee
            LIMIT 10 """).fetchall()

[('Hove', 0.5),
 ('Linkeroever', 0.9),
 ('Berlare', 1.0),
 ('Boechout', 1.0),
 ('Buggenhout', 1.0),
 ('De Haan', 1.0),
 ('De Klinge', 1.0),
 ('Kortenberg', 1.0),
 ('Lebbeke', 1.0),
 ('Niel', 1.0)]

In [18]:
data = cur.execute("""SELECT * FROM main.restaurants""")
df = pd.DataFrame(data.fetchall(), columns=[x[0] for x in data.description])

In [19]:
#df = df.dropna(subset=['city'])
fees = df[(df['delivery_fee'] > 0) & (df['city'].notnull())]
delivery_fees = fees.groupby('city')['delivery_fee'].mean().sort_values().reset_index()
delivery_fees.index = delivery_fees.index + 1 
delivery_fees.head(10)

,city,delivery_fee
1,Hove,0.5
2,Linkeroever,0.9
3,Boechout,1.0
4,Berlare,1.0
5,Buggenhout,1.0
6,Sint Niklaas,1.0
7,Termonde,1.0
8,Waasmunster,1.0
9,Niel,1.0
10,Lebbeke,1.0


In [39]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(delivery_fees.columns),
                fill_color='lightblue',
                align='center',),
    cells=dict(values=[delivery_fees[col].head(10) for col in delivery_fees.columns],
               fill_color='#87bba2',
               align='center')),
               
])

fig.update_layout(
    width=400,  
    height=410,
    title = "Cities with lowest average delivery fee" 
)
# Show the table
fig.show()

- Map locations offering kapsalons and their average price.

In [21]:
kapsalon = """
SELECT rest_name, latitude, longitude, ROUND(AVG(price),2) AS avg_ks_price
FROM main.restaurants
WHERE item_name LIKE '%Kapsalon%'
GROUP BY rest_id
ORDER BY rest_name;
   """

In [22]:
kapsalon_df = pd.DataFrame(cur.execute(kapsalon).fetchall(), columns= [x[0] for x in cur.execute(kapsalon).description])

In [23]:
kapsalon_data = cur.execute(kapsalon).fetchall()

rest_names = [row[0] for row in kapsalon_data]
latitudes = [row[1] for row in kapsalon_data]
longitudes = [row[2] for row in kapsalon_data]
avg_kp_price = [row[3] for row in kapsalon_data]

In [24]:
# Create the map
fig = go.Figure(go.Scattermapbox(
    lat=latitudes,
    lon=longitudes,
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=10,
        color=avg_kp_price,  
        colorscale='Viridis',  
        showscale=True  
    ),
    text=[f"{name}<br>Avg. kapsalon price: €{price}" for name, price in zip(rest_names, avg_kp_price)],  # Hover text
    hoverinfo='text'
))

fig.update_layout(
    mapbox=dict(
        accesstoken='your_mapbox_access_token', 
        style="carto-positron",  # Map style
        center=dict(lat=50.8503, lon=4.3517),
        zoom=7.2  
    ),
    title=dict(
        text="Restaurants in Belgium offering kapsalon",
        font=dict(size=20),  # Adjust font size
        x=0.5  # Center the title
    ),
    margin={"r":0,"t":50,"l":0,"b":0}  # Add space at the top for the title
)

# Show the map
fig.show()

In [25]:
print("Tables in the new database:")
for row in conn.execute("SELECT name FROM main.sqlite_master WHERE type='table';"):
    print(row)

Tables in the new database:
('restaurants',)
